In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from pathlib import Path
import re

In [25]:
base_file_path="C:/Users/rajas/OneDrive - Indiana University/Desktop/Desktop/Applied Data Science/H501_video_game_project/h501-group6/igdb_data/"

In [26]:
def load_csvs_prefix_before_numbers(folder):
    """
    Reads all CSVs in `folder` and returns a dict of DataFrames keyed by the
    filename prefix (all tokens before the first all-numeric token).
    Examples:
      'igdb_game_time_to_beats_20250925_1112.csv' -> 'igdb_game_time_to_beats'
      'sales_orders_2024_01.csv' -> 'sales_orders'
    If no numeric token exists, the full stem is used.
    Name collisions get suffixes _2, _3, ...
    """
    folder = Path(folder)
    dfs = {}
    for fp in sorted(folder.glob("*.csv")):
        stem = fp.stem  # e.g., 'igdb_game_time_to_beats_20250925_1112'
        tokens = stem.split("_")

        # Find first purely numeric token; take everything before it
        prefix_tokens = []
        for t in tokens:
            if re.fullmatch(r"\d+", t):
                break
            prefix_tokens.append(t)

        prefix = "_".join(prefix_tokens) if prefix_tokens else stem

        # sanitize for a safe variable-like key
        safe = "".join(ch if (ch.isalnum() or ch == "_") else "_" for ch in prefix) or "df"

        # ensure uniqueness
        name = safe
        i = 2
        while name in dfs:
            name = f"{safe}_{i}"
            i += 1

        try:
            df = pd.read_csv(fp)
        except Exception as e:
            print(f"Failed to read {fp.name}: {e}")
            continue

        dfs[name] = df
        print(f"Loaded {fp.name} → dfs['{name}'] (shape={df.shape})")

    return dfs

Read the bridge tables

In [49]:
dfs = load_csvs_prefix_before_numbers(base_file_path)

Loaded igdb_external_game_sources_20250930_1816.csv → dfs['igdb_external_game_sources'] (shape=(22, 5))
Loaded igdb_game_time_to_beats_20250925_1112.csv → dfs['igdb_game_time_to_beats'] (shape=(6092, 9))
Loaded igdb_games_20250930_1628.csv → dfs['igdb_games'] (shape=(336751, 40))
Loaded igdb_genres_20250925_1122.csv → dfs['igdb_genres'] (shape=(23, 7))
Loaded igdb_platform_families_20250925_1129.csv → dfs['igdb_platform_families'] (shape=(5, 4))
Loaded igdb_platform_logos_20250925_1134.csv → dfs['igdb_platform_logos'] (shape=(389, 8))
Loaded igdb_platform_types_20250925_1135.csv → dfs['igdb_platform_types'] (shape=(6, 5))
Loaded igdb_platform_versions_20250925_1136.csv → dfs['igdb_platform_versions'] (shape=(422, 9))
Loaded igdb_platforms_20250925_1127.csv → dfs['igdb_platforms'] (shape=(220, 16))
Loaded igdb_popularity_primitives_20250925_1153.csv → dfs['igdb_popularity_primitives'] (shape=(222990, 9))
Loaded igdb_popularity_types_20250925_1126.csv → dfs['igdb_popularity_types'] (shap

In [50]:
df_time_to_beat=dfs.get("igdb_game_time_to_beats")
df_games=dfs.get("igdb_games")
df_genres=dfs.get("igdb_genres")
df_platform_families=dfs.get("igdb_platform_families")
df_platform_logos=dfs.get("igdb_platform_logos")
df_platform_types=dfs.get("igdb_platform_types")
df_platform_versions=dfs.get("igdb_platform_versions")
df_platforms=dfs.get("igdb_platforms")
df_popularity_primitives=dfs.get("igdb_popularity_primitives")
df_popularity_types=dfs.get("igdb_popularity_types")
df_external_game_sources=dfs.get("igdb_external_game_sources")

In [51]:
df_time_to_beat.columns

Index(['id', 'game_id', 'hastily', 'normally', 'completely', 'count',
       'created_at', 'updated_at', 'checksum'],
      dtype='object')

In [52]:
all_games=df_games['id'].unique()

In [53]:
df_ttb=df_time_to_beat[df_time_to_beat['game_id'].isin(all_games)]
df_ttb.shape

(6088, 9)

In [54]:
df_ttb.head()

,id,game_id,hastily,normally,completely,count,created_at,updated_at,checksum
0,1,873,63720.0,93360.0,134748.0,7,1728556530,1729799010,f2c2039a-9cef-dfe6-0450-7d9a5ae6a9a7
1,2,56,90000.0,291360.0,1116000.0,5,1728556530,1728556530,cce80fb9-8521-449d-717d-acad7fd64ab0
2,3,2,30200.0,68100.0,111150.0,6,1728556530,1728556530,b2668ba1-27ca-58e8-70df-9672c0c2ff6a
3,4,6279,NaN,22320.0,NaN,1,1728556530,1728556530,174639b5-f6cf-1a1a-2cf2-8f0999664774
4,5,494,26700.0,29820.0,32100.0,2,1728556530,1746130808,7d5d94cb-f25e-ab83-ce11-5f386a819449


In [60]:
name_map = df_games.set_index('id')['name']
pop_map=df_popularity_types.set_index('id')['name']
pop_source_map=df_external_game_sources.set_index('id')['name']
df_popularity_primitives['game_name'] = df_popularity_primitives['game_id'].map(name_map)
df_popularity_primitives['popularity_type_name']=df_popularity_primitives['popularity_type'].map(pop_map)
df_popularity_primitives['popularity_source_name']=df_popularity_primitives['external_popularity_source'].map(pop_source_map)

In [63]:
df_pop_prim=df_popularity_primitives[['id','game_id','game_name','popularity_type_name','popularity_source_name','value','calculated_at']]

In [64]:
df_pop_prim.head()

,id,game_id,game_name,popularity_type_name,popularity_source_name,value,calculated_at
0,1,1,Thief II: The Metal Age,Playing,IGDB,0.000066,1758762234
1,2,2,Thief: The Dark Project,Playing,IGDB,0.000033,1758762234
2,3,3,Thief: Deadly Shadows,Playing,IGDB,0.000066,1758762234
3,4,4,Thief,Playing,IGDB,0.000243,1758762234
4,5,5,Baldur's Gate,Playing,IGDB,0.000155,1758762234


In [67]:
df_games.head()

,id,age_ratings,aggregated_rating,aggregated_rating_count,alternative_names,artworks,bundles,cover,created_at,external_games,...,total_rating,total_rating_count,updated_at,url,videos,websites,checksum,language_supports,collections,game_type
0,1,123634|129407|5855|123633,90.000000,1.0,49639|49640|3444|3442|3443|20595,95423,53796|275496,291911.0,1297556438,149929|73551|37911|246583|13083|78299,...,88.587045,143.0,1759029049,https://www.igdb.com/games/thief-ii-the-metal-age,522|35104,119762|62678|19688|19689|333803,ab7cf468-78a4-592e-7c9c-d4610acfd68b,19414|476623|476624|476622,2,0
1,2,5854|177340|30959|176894,70.000000,1.0,3437|3436|3438|3439|3440|3441,122795,NaN,96746.0,1297561968,247128|38915|152898,...,78.316985,165.0,1759001022,https://www.igdb.com/games/thief-the-dark-project,524|131421|131422|131423,144265|144266|334329|144264,8cefbdca-b443-880a-0781-ac15ce5e118f,NaN,2,0
2,3,3|123843|123842|32546,83.000000,2.0,41044|41045|3448|3446|3445|3447|82231|82232,NaN,53796|275496,129190.0,1297562649,91563|246113|154888|78300|14910,...,82.448155,125.0,1758683678,https://www.igdb.com/games/thief-deadly-shadows,523,40859|40858|104881|104880|119776|333347,01a2d40e-c521-5a5d-f825-5349f767c2a3,28309|477853|477854|477855|477856,2,0
3,4,111505|111506|94708|95289|187774,63.642857,14.0,14529|14528|14530|46097|46096|46098|181157,4789|168685|168686|7197|4796|4790|4788|4787|47...,53796|56723|93888|154657,96744.0,1297562870,1935791|245922|83900|62671|91560|401830|123815...,...,66.771327,333.0,1758866890,https://www.igdb.com/games/thief,1939|1458|25769|25770|25776|141702,78109|146884|95317|880085|800068|47846|94900|7...,e55d02d9-e396-b8bd-02ac-d09bd2f2cdf1,201849|201828|201829|201830|201831|201832|2018...,2,0
4,5,28012|28013|80725,NaN,NaN,68625|183373,NaN,141859,93899.0,1297562972,246037|37729|225253|156957,...,85.800206,324.0,1759247568,https://www.igdb.com/games/baldur-s-gate,246,268715|333273,04621639-86b9-6318-1124-9baaa1437787,NaN,7,0
